In [0]:
%sql
select *
from sandbox.t_weetv.po_activation_mart_v2

In [0]:
%sql
select *
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and Act_Date is null

In [0]:
%sql
select 
  max(PO_Date), max(Act_Date), max(he_etl_dt)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  -- and Act_Date is null

In [0]:
%sql
select 
  max(PO_Date), max(Act_Date), max(he_etl_dt)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  -- and Act_Date is null

In [0]:
%sql
select 
  PO_date_ym, count(1), count(distinct MAC)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM or PO_Brand != Act_Brand)
group by
  PO_date_ym

In [0]:
%sql
select 
  count(1), count(distinct MAC)
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  -- and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM)
-- group by
--   PO_date_ym

In [0]:
%sql
select *
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM or PO_Brand != Act_Brand)

In [0]:
%sql
select 
  mac_addr as MAC_ORIGIN, eic_data_ods.tlamp.hash_mac(mac_addr) as MAC
from eic_data_private.tlamp.activation_date

In [0]:
%sql
create or replace view sandbox.z_yeswook_kim.rbt_mac
AS
(
select w60.*, 'webos60' as webos_version
from eic_data_private.tlamp.rbt_mac_addr_webos60 w60
union all
select w22.*, 'webos22' as webos_version
from eic_data_private.tlamp.rbt_mac_addr_webos22 w22
union all
select w23.*, 'webos23' as webos_version
from eic_data_private.tlamp.rbt_mac_addr_webos23 w23
union all
select w24.*, 'webos24' as webos_version
from eic_data_private.tlamp.rbt_mac_addr_webos24 w24
union all
select w25.*, 'webos25' as webos_version
from eic_data_private.tlamp.rbt_mac_addr_webos25 w25
)

In [0]:
%sql
select webos_version, count(1), count(distinct mac_addr)
from sandbox.z_yeswook_kim.rbt_mac
group by webos_version

In [0]:
%sql
select 
  count(1)
  , count(distinct concat(mac_addr,mac_addr_hashed, webos_version))
  , count(distinct concat(mac_addr,mac_addr_hashed))
  , count(distinct concat(mac_addr_hashed))
from sandbox.z_yeswook_kim.rbt_mac
-- group by webos_version

In [0]:
%sql
with tb0 as (
select *
from sandbox.t_weetv.v_po_activation_detail
where 1=1
  and PO_Date>='2023-01-01'
  -- and Act_Date>='2023-01-01'
  and (PO_Board_Maker != Act_Board_Maker 
    or PO_ODM != Act_ODM)
), tb1 as (
  select 
    mac_addr as MAC_ORIGIN
    , mac_addr_hashed as MAC
  from sandbox.z_yeswook_kim.rbt_mac
  union all 
  select 
     mac_addr as MAC_ORIGIN
     , eic_data_ods.tlamp.hash_mac(mac_addr) as MAC
  from eic_data_private.tlamp.activation_date
), tb2 as (
  select 
    distinct MAC_ORIGIN, MAC
  from tb1
)
select *
from (
  select 
    tb2.mac_origin
    , tb0.*
    , DATEDIFF(tb0.Act_Date, tb0.PO_Date) AS day_diff
    , row_number() over (order by PO_date_ym, PO_Date, tb2.mac_origin) as rn
  from tb0 
  left join tb2 USING (MAC)
) t
where rn between 1 and 800000
-- where rn between 1600001 and 2400000
order by PO_date_ym, PO_Date, mac_origin